#ML Name Matching with Policy Data

In [97]:
import pandas as pd
import numpy as np


In [98]:
#Import data from 2018-19
df = pd.read_csv(r"P:\\Consulting\\Data\\Education\\EdDataExpress\\18-19\\District\\All students\\LEA_Performance_18-19_WA_CA_TX_PA.csv")
dfall = df.copy()

In [99]:
#Keep identifying variables
df = df[['State','NCES LEA ID','LEA']].reset_index(drop=True)

In [100]:
#Make columns lowercase
df.columns = map(str.lower, df.columns)
#Make values lowercase
for col in df[['state','lea']]:
    df[col] = df[col].str.lower()

In [101]:
#Load policy data
dir = r"C:\\Users\\HavalaHanson\\OneDrive - Evergreen Analytics\\Racially Just Schools\\WT Grant\\"
df_policy = pd.read_excel(dir + "\\Policy Components.xlsx", sheet_name="Complete Sheet")
df_policy.columns = map(str.lower, df_policy.columns)
#Strip whitespace from column names
df_policy.columns = df_policy.columns.str.strip()

df_policy = df_policy[['district name', 'location', 'year', 'document type 👇🏾 (drop down)','definitions',
       'district mission, vision, core values', 'stated actions for change',
       'stated commitments', 'intended outcomes/goals',
       'policy purpose statement', 'institutional responsibility',
       'usdoe effective policy components total',
       'stated position on racial/equity issues',
       'enumeration of racialized groups',
       'naming racism and/or other forms of racism',
       'race and racism consciouness components total:',
       'redistribution of power', 'redistribution of resources',
       'critical policy analysis components total', 'total score 👆🏾']]

#Remove emojis from column names
df_policy.columns = df_policy.columns.str.replace('👇🏾','').str.replace('👆🏾','')
#Strip whitespace from column names
df_policy.columns = df_policy.columns.str.strip()

In [102]:
#Drop if missing Year
df_policy = df_policy.dropna(subset=['year'])
#Drop if policy is nd
df_policy = df_policy[df_policy['year'] != 'nd'].reset_index(drop=True)

#Make a copy of the policy data
df_policy_all = df_policy.copy()

In [103]:
#Keep district name and location
df_policy = df_policy[['district name', 'location']].reset_index(drop=True)

In [104]:
#Make all columns lowercase
df_policy.columns = map(str.lower, df_policy.columns)
#Make all values lowercase
for col in df_policy.columns:
    df_policy[col] = df_policy[col].str.lower()

In [105]:
#Keep only CA, TX, PA, and WA
df_policy = df_policy[df_policy['location'].isin(['california','texas','pennsylvania','washington'])].reset_index(drop=True)

#Drop duplicates
df_policy = df_policy.drop_duplicates().reset_index(drop=True)


In [106]:
#California school for the blind often erroneously matches. Remove it from the df dataframe
df = df[~df['lea'].str.contains('blind')].reset_index(drop=True)
df = df[~df['lea'].str.contains(' deaf')].reset_index(drop=True)

In [ ]:
#Perform fuzzy merge using rapidfuzz
from rapidfuzz import process, fuzz

#Remove school and district from the district name
df_policy['orig_district_name'] = df_policy['district name']
df_policy['district name'] = df_policy['district name'].str.replace('schools','').str.replace('school','').str.replace('district','').str.replace(' sd','').str.replace('elementary','').str.replace('middle ','').str.replace('academy','').str.replace(' public','').str.replace('joint','').str.replace('county','').str.replace(' area','').str.replace('regional','').str.replace('office','').str.replace('educational','').str.replace('education','').str.replace('services','').str.replace('education','')
df['lea'] = df['lea'].str.replace('schools','').str.replace('school','').str.replace('district','').str.replace(' sd','').str.replace('elementary','').str.replace('middle ','').str.replace('academy','').str.replace(' public','').str.replace('joint','').str.replace('county','').str.replace(' area','').str.replace('regional','').str.replace('office','').str.replace('educational','').str.replace('education','').str.replace('services','').str.replace('education','')

#Replace independent with isd
df['lea'] = df['lea'].str.replace('independent','isd')
df_policy['district name'] = df_policy['district name'].str.replace('independent','isd')

#Trim lea and district name
df['lea'] = df['lea'].str.strip()
df_policy['district name'] = df_policy['district name'].str.strip()

#Create a dictionary to store the matches
matches = pd.DataFrame(columns=['state','district name','lea','ratio','id'])
m0 = {}
m1 = {}

#Set seed
np.random.seed(123)

#Iterate over each state
for state in df_policy['location'].unique():
    #Subset the data
    df_policy_state= df_policy[df_policy['location'] == state].reset_index(drop=True)
    df_state = df[df['state'] == state].reset_index(drop=True)
    #Iterate over each district
    for i in range(len(df_policy_state)):
        #Get the district name
        district = df_policy_state.loc[i,'district name']
        #Get the best match
        match = process.extract(district, df_state['lea'])
        #Add the match to the dictionary
        m0[district] = match[0]
        m1[district] = match[1]
        #Add state to the dictionary
        m0[district] = (m0[district][0],m0[district][1],
                        df_state[df_state['lea'] == m0[district][0]]['nces lea id'].values[0])
        m1[district] = (m1[district][0],m1[district][1],
                        df_state[df_state['lea'] == m1[district][0]]['nces lea id'].values[0])
        #Create a dataframe from the dictionary
        df_matches = pd.DataFrame(list(m0.items()), columns=['district name','lea'])
        df_matches1 = pd.DataFrame(list(m1.items()), columns=['district name','lea'])
        #Divide the lea column into three columns
        df_matches[['lea','ratio','id']] = pd.DataFrame(df_matches['lea'].tolist(), index=df_matches.index)
        df_matches1[['lea','ratio','id']] = pd.DataFrame(df_matches1['lea'].tolist(), index=df_matches.index)
        

#Merge df_matches with df_matches1
df_matches = df_matches.merge(df_matches1, on='district name', how='left')

#Create true match column 
df_matches['true_match'] = df_matches['lea_x']
df_matches['true_match'] = np.where(df_matches['district name'] == df_matches['lea_y'], df_matches['lea_y'], df_matches['true_match'])

#Assert that lea_x is equal to true_match
assert df_matches['lea_x'].equals(df_matches['true_match'])

#Drop _y columns
df_matches = df_matches.drop(columns=['lea_y','ratio_y','id_y']).rename(columns={'lea_x':'lea','ratio_x':'ratio','id_x':'id'})
#Drop true_match column
df_matches = df_matches.drop(columns=['true_match'])

#Assert no duplicates
assert df_matches.duplicated().sum() == 0

In [108]:
#Merge with df_policy to get full district name. only keep columns needed
df_matches = df_matches.merge(df_policy, on='district name', how='left')

In [109]:
#Create indicators for policy and resolution
df_policy_all['document type  (drop down)'].value_counts()

#Lowercase values 
df_policy_all['document type  (drop down)'] = df_policy_all['document type  (drop down)'].str.lower()
#Assert not missing document type
assert df_policy_all['document type  (drop down)'].isnull().sum() == 0
df_policy_all['policy'] = np.where(df_policy_all['document type  (drop down)'].str.contains('policy'), 1, 0)
df_policy_all['resolution'] = np.where(df_policy_all['document type  (drop down)'].str.contains('resolution'), 1, 0)

In [110]:
#Replace year with 2021 if year is 2012 or 2021
df_policy_all['year'] = np.where(df_policy_all['year']=='2012 or 2021', '2021', df_policy_all['year'])

#Convert year to numeric
df_policy_all['year'] = pd.to_numeric(df_policy_all['year'], errors='coerce')

In [111]:
#Create a sample variable equal to 1 if the year is 2020 or 2021 and the document type is policy
df_policy_all['sample'] = np.where((df_policy_all['year'] >= 2020) & (df_policy_all['policy'] == 1), 1, 0)
#Create a sample variable equal to 1 if the year is 2020 or 2021 and the document type is resolution
df_policy_all['resolution_only'] = np.where((df_policy_all['year'] >= 2020) & (df_policy_all['resolution'] == 1), 1,0)

In [112]:
#Lowercase district name 
df_policy_all['district name'] = df_policy_all['district name'].str.lower()

#Merge df_policy_all with df_matches
df_policy_all = df_policy_all.merge(df_matches, left_on='district name', right_on='orig_district_name',how='left')
#Remove orig_district_name and location_y
df_policy_all = df_policy_all.drop(columns=['orig_district_name','location_y','district name_y','ratio'])
#Rename location_x to location
df_policy_all = df_policy_all.rename(columns={'district name_x':'district name','location_x':'state','district_name_x':'district name','id':'ncesid',
                                              'document type  (drop down)':'policy type'})


In [113]:
#Keep only wa, ca, tx, and pa
df_policy_all = df_policy_all[df_policy_all['state'].isin(['California','Texas','Pennsylvania','Washington'])].reset_index(drop=True)

In [114]:
#Save the data
df_policy_all.to_csv(dir + "\\Policy Components Matched.csv", index=False)